In [2]:
import sys
sys.path.append("/home/zhenhao/tc-benchmark") 

In [3]:
from tc_benchmark_util.taxid_util import *

import pandas as pd

metadata_df = pd.read_csv("/home/zhenhao/metadata_with_taxid.csv", index_col=0)
metadata_df

/tmp/ipykernel_281388/2356047776.py:5: DtypeWarning: Columns (62,64,66,75,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  metadata_df = pd.read_csv("/home/zhenhao/metadata_with_taxid.csv", index_col=0)


,accession,ambiguous_bases,checkm_completeness,checkm_contamination,checkm_marker_count,checkm_marker_lineage,checkm_marker_set_count,checkm_strain_heterogeneity,coding_bases,coding_density,...,total_gap_length,trna_aa_count,trna_count,trna_selenocysteine_count,genus_name,family_name,species_name,genus_taxid,species_taxid,family_taxid
0,RS_GCF_001206855.1,14,99.60,0.91,761,g__Streptococcus (UID584),238,50.00,1827337,86.246727,...,21,19,35,0,Streptococcus,Streptococcaceae,Streptococcus pneumoniae,1301.0,1313,1300.0
1,RS_GCF_001108865.1,13,99.39,0.34,761,g__Streptococcus (UID584),238,33.33,1852015,86.277715,...,0,19,38,0,Streptococcus,Streptococcaceae,Streptococcus pneumoniae,1301.0,1313,1300.0
2,RS_GCF_000730125.1,30,100.00,0.92,476,f__Bifidobacteriaceae (UID1462),217,0.00,2212698,86.271453,...,619,19,55,0,Bifidobacterium,Bifidobacteriaceae,Bifidobacterium infantis,1678.0,216816,31953.0
3,RS_GCF_000950275.1,0,99.45,0.53,921,f__Vibrionaceae (UID4865),366,0.00,4128151,84.994806,...,0,17,77,1,Photobacterium,Vibrionaceae,Photobacterium angustum,657.0,661,641.0
4,RS_GCF_000468955.1,0,100.00,0.00,472,o__Lactobacillales (UID543),265,0.00,2107260,86.823994,...,0,19,60,0,Lactococcus,Streptococcaceae,Lactococcus cremoris,1357.0,1359,1300.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394927,GB_GCA_934776845.1,0,91.57,0.00,149,k__Bacteria (UID2329),89,0.00,779596,89.630474,...,0,19,41,0,UMGS687,UBA1242,UMGS687 sp900544595,NaN,244328,NaN
394928,GB_GCA_023647245.1,4609,99.97,0.13,1162,f__Enterobacteriaceae (UID5121),336,0.00,4707023,87.866887,...,0,19,80,1,Klebsiella,Enterobacteriaceae,Klebsiella pneumoniae,570.0,573,543.0
394929,GB_GCA_020344015.1,0,93.69,3.30,147,k__Bacteria (UID2495),91,0.00,4223673,87.882712,...,27200,18,46,1,G020344215,KS3-K002,G020344215 sp020344015,NaN,2026742,NaN
394930,GB_GCA_945889495.1,0,90.94,3.23,247,c__Deltaproteobacteria (UID3216),155,0.00,4966708,87.941933,...,0,18,46,1,DP-20,UBA9968,DP-20 sp945889495,29526.0,2043168,213465.0


In [28]:
from tqdm import tqdm

def parse_sam_file(sam_file):
    classifications = []
    last_read_id = None
    read_map_result = {}
    with open(sam_file) as f:
        for line in tqdm(f.readlines()):
            if line[0] == "@":
                continue

            # parse the line
            map_result = line.split("\t")
            read_id = map_result[0]
            if read_id != last_read_id:
                # record the classification of the last read
                if last_read_id is not None:
                    if len(read_map_result) == 0:
                        classifications.append(0)
                    else:
                        # find the taxid with the highest map quality
                        best_taxid = max(read_map_result, key=read_map_result.get)
                        classifications.append(int(best_taxid))
                
                # reset the read_map_result
                read_map_result = {}
            
            # update the read_map_result
            if map_result[2] != "*":
                taxid = map_result[2].split("|")[-1]
                map_quality = int(map_result[4])
                read_map_result[taxid] = map_quality
            
            # update the last_read_id
            last_read_id = read_id
        
        # record the classification of the last read
        if len(read_map_result) == 0:
            classifications.append(0)
        else:
            # find the taxid with the highest map quality
            best_taxid = max(read_map_result, key=read_map_result.get)
            classifications.append(int(best_taxid))
            
    return classifications

In [32]:
def check_read_level_predictions(alignment_results, ground_truth):
    """
    Return a dataframe containing Kraken's results and the sensitivity.
    """
    result = {}

    # For each level, check if the classification is correct
    for rank in ['strain', 'species', 'genus', 'family']:
        total = 0
        correct = 0
        mapped = 0
        for i in range(len(ground_truth)):
            # Check if Kraken2 classified at this level
            mapped += (alignment_results[i] != 0)
            
            # Check if the classification is correct
            if rank == 'strain':
                correct += (alignment_results[i] == ground_truth[i])
            else:
                correct += check_equivalence(alignment_results[i], ground_truth[i], rank)
            
            total += 1
        
        result[rank + "_sensitivity"] = correct / total if total > 0 else 0
        result[rank + "_precision"] = correct / mapped if mapped > 0 else 0

    return result

In [33]:
import glob
ground_truth_files = "/mnt/c/Users/zhenh/tax_reads/train_reads.label"
kraken2_reports = glob.glob("/mnt/c/Users/zhenh/minimap2_benchmark/query0_ref0.output")

with open(ground_truth_files) as f:
    ground_truth = [int(line.strip()) for line in f.readlines()]

sensitivity = {}
precision = {}
f1 = {}

for rank in ['strain', 'species', 'genus', 'family']:
    sensitivity[rank] = []
    precision[rank] = []
    f1[rank] = []


for report in kraken2_reports:
    alignment_results = parse_sam_file(report)
    results = check_read_level_predictions(alignment_results, ground_truth)
    # Store the results
    for rank in ['strain', 'species', 'genus', 'family']:

        sensitivity[rank] = sensitivity[rank] + [results[rank + "_sensitivity"]]
        precision[rank] = precision[rank] + [results[rank + "_precision"]]
        f1[rank] = f1[rank] + [2 * results[rank + "_sensitivity"] * results[rank + "_precision"] / (results[rank + "_sensitivity"] + results[rank + "_precision"])]

print(sensitivity)
print(precision)
print(f1)


100%|██████████| 239857/239857 [00:00<00:00, 801938.28it/s] 


{'strain': [0.8476584994334331], 'species': [0.8994854457303141], 'genus': [0.9701484219716531], 'family': [0.9814054576189326]}
{'strain': [0.8495364337044345], 'species': [0.9014781993521243], 'genus': [0.9722977249878989], 'family': [0.9835796998920207]}
{'strain': [0.8485964276083946], 'species': [0.9004807200572772], 'genus': [0.9712218843854316], 'family': [0.9824913758635759]}


In [26]:
len(alignment_results)

53832

In [27]:
len(ground_truth)

53833